In [1]:
# ==========================================
# Housing Data - Final Cleaning Pipeline
# ==========================================
# This notebook takes raw data and produces clean, modeling-ready data
# Team members: Hamid Iqbal, Ibrahim Afkir
# What this does: Loads raw CSV, cleans columns, creates features, saves clean data
# ==========================================

import pandas as pd
import numpy as np
from pathlib import Path

# ==========================================
# 1. Load Raw Data (1M random sample, reproducible)
# ==========================================
print("📂 Loading raw housing data...")
csv_path = Path("Dataset_1_UK_housing/price_paid_records.csv")

total_rows = 22_489_348
sample_size = 1_000_000

np.random.seed(42)  # ✅ reproducible sampling

print(f"Loading random {sample_size:,} records from {total_rows:,} total...")
skip = sorted(np.random.choice(range(1, total_rows), total_rows - sample_size, replace=False))
df_raw = pd.read_csv(csv_path, skiprows=skip)

print(f"✅ Loaded {len(df_raw):,} records")
print(f"Columns: {df_raw.columns.tolist()}")

# ==========================================
# 2. Rename Columns
# ==========================================
print("\n🔧 Renaming columns...")

column_mapping = {
    "Transaction unique identifier": "transaction_id",
    "Price": "price",
    "Date of Transfer": "transfer_date",
    "Property Type": "property_type",
    "Old/New": "is_new",
    "Duration": "tenure_type",
    "Town/City": "town_city",
    "District": "district",
    "County": "county",
    "PPDCategory Type": "ppd_category",
    "Record Status - monthly file only": "record_status",
    "Postcode": "postcode",
    "PAON": "paon",
    "SAON": "saon",
    "Street": "street",
    "Locality": "locality",
}

df = df_raw.rename(columns=column_mapping)
print("✅ Columns renamed")

# ==========================================
# 3. Fix Data Types & Date Features
# ==========================================
print("\n🔧 Converting data types...")

df["transfer_date"] = pd.to_datetime(df["transfer_date"])

df["year"] = df["transfer_date"].dt.year
df["month"] = df["transfer_date"].dt.month
df["quarter"] = df["transfer_date"].dt.quarter
df["day_of_week"] = df["transfer_date"].dt.dayofweek

print("✅ Date features created")

# ==========================================
# 4. Create Label Columns
# ==========================================
print("\n🔧 Creating categorical labels...")

property_type_mapping = {
    "D": "Detached",
    "S": "Semi-Detached",
    "T": "Terraced",
    "F": "Flat",
    "O": "Other",
}
df["property_type_label"] = df["property_type"].map(property_type_mapping)

df["is_new_build"] = df["is_new"] == "Y"

df["tenure_label"] = df["tenure_type"].map({"F": "Freehold", "L": "Leasehold"})
df["tenure_label"] = df["tenure_label"].fillna("Unknown")

print("✅ Labels created")

# ==========================================
# 5. Clean Text Fields
# ==========================================
print("\n🔧 Cleaning text fields...")

text_cols = ["town_city", "district", "county", "postcode", "locality", "street"]

for col in text_cols:
    if col in df.columns:
        df[col] = df[col].fillna("UNKNOWN").astype(str).str.strip().str.upper()

print("✅ Text fields cleaned")

# ==========================================
# 6. Basic sanity checks (price & missing)
# ==========================================
print("\n🔍 Cleaning basic issues...")

# Remove non-positive prices (just in case)
before_price = len(df)
df = df[df["price"] > 0]
print(f"✅ Removed {before_price - len(df):,} rows with non-positive price")

# Show remaining missing values
missing = df.isna().sum()
missing = missing[missing > 0]
if len(missing) == 0:
    print("✅ No remaining missing values (after our cleaning)")
else:
    print("⚠️ Remaining missing values:")
    print(missing)

# ==========================================
# 7. Drop columns we don't want for modeling
# ==========================================
print("\n🔧 Dropping unnecessary / duplicate columns...")

cols_to_drop = [
    "ppd_category",
    "record_status",
    "transaction_id",
    "paon",
    "saon",
    "street",
    "locality",
    "transfer_date",      # we keep year/month/quarter/day_of_week instead
    "property_type",      # keep property_type_label
    "is_new",             # keep is_new_build
    "tenure_type",        # keep tenure_label
]

df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")

print("✅ Unnecessary columns removed")

# Remove unrealistic low prices
before_low = len(df)
df = df[df["price"] >= 10000]   # keep prices >= £10k
print(f"Removed {before_low - len(df):,} rows below £10k")

# ==========================================
# 8. Remove Extreme Outliers (3-sigma)
# ==========================================
print("\n🔧 Removing outliers using 3-sigma rule...")

mean_price = df["price"].mean()
std_price = df["price"].std()

lower_bound = max(0, mean_price - 3 * std_price)
upper_bound = mean_price + 3 * std_price

print(f"  Mean: £{mean_price:,.0f}")
print(f"  Std Dev: £{std_price:,.0f}")
print(f"  Lower bound: £{lower_bound:,.0f}")
print(f"  Upper bound: £{upper_bound:,.0f}")

before_outliers = len(df)
df = df[(df["price"] >= lower_bound) & (df["price"] <= upper_bound)].copy()
print(f"  Original records: {before_outliers:,}")
print(f"  After filtering: {len(df):,}")
print(f"  Removed: {before_outliers - len(df):,} extreme outliers")


# ==========================================
# 9. Final Summary
# ==========================================
print("\n📊 FINAL CLEAN DATASET SUMMARY:")
print("=" * 50)
print(f"Total records: {len(df):,}")
print(f"Columns: {len(df.columns)}")
print(f"Years: {df['year'].min()} - {df['year'].max()}")
print(f"Price range: £{df['price'].min():,} - £{df['price'].max():,}")
print("\nColumns:")
print(df.columns.tolist())

# ==========================================
# 10. Save Clean Data
# ==========================================
print("\n💾 Saving clean data...")
output_path = Path("data/cleaned/housing_clean.parquet")
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_parquet(output_path, index=False)
print(f"✅ Saved to {output_path}")
print(f"File size: {output_path.stat().st_size / (1024**2):.1f} MB")

print("\n🎉 CLEANING COMPLETE!")
print(f"\n💡 This file can now be used by the housing deployment:")
print(f"   - housing-deployment/frontend will use it for dropdown options")
print(f"   - Location: {output_path.absolute()}")


📂 Loading raw housing data...
Loading random 1,000,000 records from 22,489,348 total...
✅ Loaded 1,000,000 records
Columns: ['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']

🔧 Renaming columns...
✅ Columns renamed

🔧 Converting data types...
✅ Loaded 1,000,000 records
Columns: ['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']

🔧 Renaming columns...
✅ Columns renamed

🔧 Converting data types...
✅ Date features created

🔧 Creating categorical labels...
✅ Labels created

🔧 Cleaning text fields...
✅ Date features created

🔧 Creating categorical labels...
✅ Labels created

🔧 Cleaning text fields...
✅ Text fields cleaned

🔍 Cleaning basic issues...
✅ Text fields cleaned

🔍 Cleaning basic issues...
✅ Remove